
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': '../_Data/8-Data_AIDataset',
 'MODEL_TASK': '../_Model',
 'WORKSPACE_PATH': 'g:\\Shared '
                   'drives\\CDHAI-WellDoc\\2024-WellDocTest-SPACE\\_WellDoc-AI-CGMGPT-WorkSpace'}


# Trigger Events

In [2]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

base_config = load_cohort_args(recfldtkn_config_path, SPACE)
print(base_config)

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

In [3]:
# CaseFolder = os.path.join(SPACE['DATA_TASK'], 'CaseFolder')
# print(CaseFolder)

# Trigger Case

In [4]:
#################
cohort_label_list = [1, 2, 3]
#################

######################################
TriggerCaseMethod = 'CGM5MinEntry'
######################################

TriggerRecName = 'CGM5Min'
case_id_columns = ["PID", "ObsDT"]
special_columns = ['PID', 'DT_s']

In [5]:

import inspect

def convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config):
    ds_case = ds_rec.select_columns(special_columns)
    df_case_raw = ds_case.to_pandas()
    
    # ------------------------------ Filter Cases ------------------------------
    df_case_filter = df_case_raw # [df_case_raw['invitation_state'] == 'Delivered'].reset_index(drop=True)
    # --------------------------------------------------------------------------

    # ------------------------------ Determine the ObsDT ------------------------------
    df_case_filter['ObsDT'] = df_case_filter['DT_s'] # [df_case_raw['drug_name'] == 'Trulicity']
    df_case_filter = df_case_filter.drop(columns = 'DT_s') # .from_pandas(df_case_filter)
    # --------------------------------------------------------------------------

    # ------------------------------- Update Column Sequence ------------------------
    columns = df_case_filter.columns 
    columns = case_id_columns + [col for col in columns if col not in case_id_columns]
    df_case_filter = df_case_filter[columns].reset_index(drop=True)

    df_case = df_case_filter
    return df_case

convert_TriggerEvent_to_Caseset.fn_string = inspect.getsource(convert_TriggerEvent_to_Caseset)

In [6]:
prefix = ['import pandas as pd', 'import numpy as np']
string_variables = [TriggerRecName]
iterative_variables = [case_id_columns, special_columns]
fn_variables = [convert_TriggerEvent_to_Caseset]
pycode = convert_variables_to_pystirng(string_variables = string_variables, 
                                       iterative_variables = iterative_variables, 
                                       fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(base_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline/fn_trigger/CGM5MinEntry.py


In [7]:
pypath = os.path.join(base_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
module = load_module_variables(pypath)

######################
TriggerRecName = module.TriggerRecName
case_id_columns = module.case_id_columns
special_columns = module.special_columns
convert_TriggerEvent_to_Caseset = module.convert_TriggerEvent_to_Caseset
######################

In [8]:
############################### ds_rec to df_case ###############################
ds_rec, _ = load_ds_rec_and_info(TriggerRecName, base_config, cohort_label_list)
df_case = convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)
df_case

,PID,ObsDT
0,1000006,2021-01-07 06:20:00
1,1000006,2021-01-07 06:25:00
2,1000006,2021-01-07 06:30:00
3,1000006,2021-01-07 06:35:00
4,1000006,2021-01-07 06:40:00
...,...,...
21215907,3000040,2023-06-15 20:25:00
21215908,3000040,2023-06-15 21:05:00
21215909,3000040,2023-06-15 21:10:00
21215910,3000040,2023-06-15 21:15:00


In [9]:
 #  = [1, 2, 3]

In [10]:
CaseFolder = os.path.join(SPACE['DATA_CaseSet'])

if not os.path.exists(CaseFolder): os.makedirs(CaseFolder)

cohort_label_list_str = '_'.join([str(i) for i in cohort_label_list])
# path = os.path.join(CaseFolder, f'{TriggerCaseMethod}-{cohort_label_list_str}.p')
path = os.path.join(CaseFolder, f'{TriggerCaseMethod}.p')
df_case.to_pickle(path)
print(path)

../_Data/4-Data_CaseSet\CGM5MinEntry.p
